In [1]:
dataset = ''
#dataset1 = dataset
from pathlib import Path
output_root = Path("result")
dirs = [fld for fld in output_root.glob(f'{dataset}-*')]

In [2]:
import pandas as pd

from lenskit import util
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from lenskit.datasets import MovieLens
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import RandomizedSearchCV
import warnings
warnings.filterwarnings('ignore')
import seaborn as sns
import matplotlib.pyplot as plt
from statsmodels.graphics.gofplots import qqplot
from scipy.stats import shapiro
from scipy.stats import friedmanchisquare, wilcoxon
from itertools import combinations
from lenskit import util

In [3]:
output_root = Path("result")

In [4]:
dirs = [fld for fld in output_root.glob(f'{dataset}-*')]

In [5]:
dirs

[]

In [6]:
pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.width', 1000)

In [7]:
# Parameters
#dataset = "CAS"
#dataset = "rbp"
#dataset = "LOG"
#output_root = Path("result")
dataset

''

In [8]:
movies = pd.read_csv(r'C:\Users\srabantiguha\lk-demo-experiment\data\ml-20m\movies.csv')
movies = movies[['movieId','genres']]
movies = movies.join(movies.pop('genres').str.get_dummies('|'))
movies.rename({'movieId': 'item'}, axis=1, inplace=True)
movies

,item,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,0,0,1,1,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,0,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3,4,0,0,0,0,0,1,0,0,1,...,0,0,0,0,0,1,0,0,0,0
4,5,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27273,131254,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
27274,131256,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
27275,131258,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
27276,131260,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
dirs = [fld for fld in output_root.glob(f'{dataset}-*')]
dirs

[]

In [10]:
def calc_new_col(row):
    
    if row <= a :
        return 'underexposed'
    
    elif row>= b:
        return 'overexposed'
    
    else:
        return 'Neutral'

In [11]:
analy = pd.DataFrame(columns = ["dataset","shape","Dist_EEL", "quantile(0.25)","quantile(0.75)","accuracy"], index= range(len(dirs)))

for p in range(len(dirs)): 
    
    #print('Dataset:-',dirs[p])
    o = dirs[p]
    df = pd.read_csv(dirs[p], sep=',')
    #print(df.shape)

    df = df[['item','difference']]
#     print('Distribution of EEL')
    print(df['difference'].describe())

    a = df.difference.quantile(0.25) 
    b = df.difference.quantile(0.75) 
#     print("a and b")
#     print(a,b)

    df["category"] = df['difference'].apply(lambda x: calc_new_col(x))

    movies1 = movies.merge(df, on = 'item', how = 'inner')
    map1 = {'underexposed':-1, 'overexposed':1, 'Neutral':0}
    movies1['category'] = movies1['category'].map(map1)
    movies1 = movies1.drop(columns=['item', 'difference'])

    X = movies1.drop(columns=['category'])
    y = movies1['category']
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.75, random_state=42)
    X_train.shape, X_test.shape
    logreg = LogisticRegression(random_state=42)
  
    logreg.fit(X_train, y_train)
    y_pred = logreg.predict(X_test)
    ac = (accuracy_score(y_test, y_pred))
    
    analy.iloc[p,0] = o
    analy.iloc[p,1] = (df.shape)
    analy.iloc[p,2] = 'Not showed'
    #analy.iloc[p,2] = df['difference'].describe()
    analy.iloc[p,3] = a
    analy.iloc[p,4] = b
    analy.iloc[p,5] = ac
    
analy1 = analy.T
analy1.columns = analy1.iloc[0]
analy1 =  analy1.drop(analy1.index[0])

In [12]:
analy1

dataset
shape
Dist_EEL
quantile(0.25)
quantile(0.75)
accuracy


In [13]:
# ml20m = MovieLens('data/ml-20m')
# ml20m.tag_genome